In [116]:
import nltk

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if not module_path in sys.path:
    sys.path.insert(0, module_path)

from innoprod.sheet_tools import get_sheet_dfs
from innoprod.wrangling.msyh_data_sharing import wrangle_roadmaps

In [117]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [118]:
data = get_sheet_dfs()
roadmaps_df = wrangle_roadmaps(data['Roadmaps'])

# Pre-processing

In [119]:
col = 'What are the internal barriers to growth? How do you intend to finance future growth? Are there sufficient leadership and management skills in the business to achieve your growth? What opportunities do you have to expand into new markets?'
new_col = col + ' [Pre-processed]'
roadmaps_df[col]

0      As with many business one of the main barriers...
1      As with many businesses, one of the main barri...
2      [REDACTED] company has sought to implement dig...
3                                                    nan
4      [REDACTED] barrier to increased adoption of di...
                             ...                        
215    As with many business one of the main barriers...
216    CRM/ERP(part), and automation â€“ automatic we...
217    [REDACTED] company has invested in implementin...
218    [REDACTED] has been very little investment in ...
219    As with many businesses, one of the main barri...
Name: What are the internal barriers to growth? How do you intend to finance future growth? Are there sufficient leadership and management skills in the business to achieve your growth? What opportunities do you have to expand into new markets?, Length: 220, dtype: str

In [120]:
from nltk.tokenize import RegexpTokenizer
phrases = [
    "[I|i]ndustry 4\\.0",
]
regexpr = "|".join(phrases) + "|[\\w']+"
tokenizer = RegexpTokenizer(regexpr)
# regexpr

In [121]:
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))

In [122]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [123]:
def preprocess(row):
    result = tokenizer.tokenize(row)
    result = [lemmatizer.lemmatize(word) for word in result if (word != 'REDACTED') and (not word.lower() in stops)]
    return result

In [124]:
roadmaps_df[new_col] = roadmaps_df.apply(lambda row: preprocess(row[col]), axis=1)
roadmaps_df[new_col]

0      [many, business, one, main, barrier, investmen...
1      [many, business, one, main, barrier, investmen...
2      [company, sought, implement, digital, technolo...
3                                                  [nan]
4      [barrier, increased, adoption, digital, techno...
                             ...                        
215    [many, business, one, main, barrier, investmen...
216    [CRM, ERP, part, automation, â, automatic, wei...
217    [company, invested, implementing, digital, tec...
218    [little, investment, industry 4.0, due, combin...
219    [many, business, one, main, barrier, investmen...
Name: What are the internal barriers to growth? How do you intend to finance future growth? Are there sufficient leadership and management skills in the business to achieve your growth? What opportunities do you have to expand into new markets? [Pre-processed], Length: 220, dtype: object